In [1]:
import os

In [2]:
%pwd

'c:\\Users\\abdul\\OneDrive\\Desktop\\ML Project [MLOPS]\\End-End-ML-Project-With-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\abdul\\OneDrive\\Desktop\\ML Project [MLOPS]\\End-End-ML-Project-With-MLflow'

In [5]:
os.environ["export MLFLOW_TRACKING_URI"] ="https://dagshub.com/Shaiksaad/End-End-ML-Project-With-MLflow.mlflow"
os.environ["export MLFLOW_TRACKING_USERNAME"] = "=Shaiksaad"
os.environ["export MLFLOW_TRACKING_PASSWORD"] = "1aa77245b10e4a3636b65a06b46e847295f0b2f1 "

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_name: Path
    all_params: dict
    metric_file: Path
    target_column: str
    ml_flow_url: str

In [7]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories, save_json

In [8]:
class configration_manager:
    def __init__(
        self,
        confing_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(confing_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            all_params=  params,
            metric_file= config.metric_file,
            target_column= schema.name,
            mlflow_url= "https://dagshub.com/Shaiksaad/End-End-ML-Project-With-MLflow.mlflow",
        )
        
        return model_trainer_config

In [9]:
import os
import joblib
import mlflow
import pandas as pd
import numpy as np
import mlflow.sklearn
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [10]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config
        
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse, mae, r2
    
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_name)
        
        test_x = test_data.drop([self.config.target_column], axis = 1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.ml_flow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            
            (rmse,mae,r2) = self.eval_metrics(test_y, predicted_qualities)
            
            scores = {"rmse": rmse, "mae":mae,"r2": r2}
            save_json(path=Path(self.config.metric_file), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metrics("rmse", rmse)
            mlflow.log_metrics("mae", mae)
            mlflow.log_metrics("r2", r2)
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.Model(model, "model", registered_model_name = "ElasticNetModel" )
            else:
                mlflow.sklearn.Model(model, "model")

In [11]:
try:
    config = configration_manager()
    model_evaluation_config  = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-08-22 11:36:39,255: INFO: common: yaml file: config\config.yaml loaded successfully!]
[2023-08-22 11:36:39,257: INFO: common: yaml file: params.yaml loaded successfully!]
[2023-08-22 11:36:39,259: INFO: common: yaml file: schema.yaml loaded successfully!]
[2023-08-22 11:36:39,260: INFO: common: created directory at: artifacts]
[2023-08-22 11:36:39,261: INFO: common: created directory at: artifacts/model_trainer]


BoxKeyError: "'ConfigBox' object has no attribute 'metric_file'"